# Voice gender recognition (DNN)

Below is the project of building voice gender recognition model(using DNN) and finding the best parameter regularization and the best accuracy.

Importing libraries

In [ ]:
!pip install dataprep

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l1, l2
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ReduceLROnPlateau
from dataprep.eda import plot

Loading the dataframe

In [ ]:
data = pd.read_csv('../input/voicegender/voice.csv')
data

We don't have a lot of objects in the dataset and I want to use a DNN. For reducing overfitting let's find the best regularization parameter and find the best accuracy then let's use DNN with Dropout. It is different techniques of regularization and interesting compare accuracies.

First of all, let's analyze the data using dataprep.eda tool

In [ ]:
plot(data)

We have equal objects labeled as female and male 

Splitting the data into train and test

In [ ]:
y = data['label'].copy()
y = np.array(y)

In [ ]:
y = np.where(y == 'male', 0, y)
y = np.where(y == 'female', 1, y)
y = np.asarray(y).astype(np.float32)

In [ ]:
x = data.drop('label', axis=1).copy()
x = np.array(x, dtype='float32')

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(x,
    y, test_size=0.25, random_state=42)

In [ ]:
regul = [0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001]
all_train, all_test = list(), list()
models = list()
INIT_LR = 0.0002
EPOCHS = 150
BATCH = 8

## Finding best regularization parameter for reducing overfitting

Let's build models with different regularization parameters.

In [ ]:
for param in regul:
    # define model
    model = Sequential()
    model.add(tf.keras.layers.Dense(input_shape=(trainX.shape[1],), units=256, 
                                activation="relu", kernel_regularizer=l2(param)))
    model.add(tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=l2(param)))
    model.add(tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=l2(param)))
    model.add(tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=l2(param)))
    model.add(tf.keras.layers.Dense(32, activation="relu", kernel_regularizer=l2(param)))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    #compile the model using Adam as an optimizer and 
    #a binary cross entropy as a loss function
    print("[INFO] training network...")
    opt = Adam(lr=INIT_LR)
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
    callbacks = [ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=0.00001)] 
    history_model = model.fit(trainX, trainY, epochs=EPOCHS, batch_size=BATCH, callbacks=callbacks, verbose=1)
    models.append(model)
    # evaluate the model using train and test data
    _, train_acc = model.evaluate(trainX, trainY, verbose=0)
    _, test_acc = model.evaluate(testX, testY, verbose=0)
    print('Param: %f, Train: %.3f, Test: %.3f' % (param, train_acc, test_acc))
    all_train.append(train_acc)
    all_test.append(test_acc)

Plotting train and test accuracies depending on regularization parameters

In [ ]:
plt.semilogx(regul, all_train, label='train', marker='o')
plt.semilogx(regul, all_test, label='test', marker='o')
plt.legend()
plt.show()

Building dataframe for finding high accuracy and low difference between train and test accuracies. It will define the best regularization parameter

In [ ]:
all_train_array = np.array(all_train)
all_test_array = np.array(all_test)
accuracy_frame = pd.DataFrame({
                        'regularization' : regul,
                        'accuracy_on_train': all_train_array*100, 
                         'accuracy_on_test': all_test_array*100,
                         'accuracy_difference': abs((all_train_array - all_test_array)*100)
                              })
acc=accuracy_frame.sort_values(by=['accuracy_difference'])
acc

The best accuracy is 97% and best parameter regularization is 0.005

In [ ]:
predict = (models[1].predict(testX) > 0.5).astype("int32")
print(classification_report(testY,predict))
print(confusion_matrix(testY, predict))

In [ ]:
print('Accuracy:',(accuracy_score(testY, predict))*100)

## DNN with Dropout

Let's build model with Dropout regularization technique for reducing overfitting

Setting the model

In [ ]:
all_train, all_test = list(), list()
models = list()
parameters = [0.1, 0.2, 0.3, 0.4, 0.5]
for i in parameters:
    model = Sequential()
    model.add(Dense(input_shape=(trainX.shape[1],), units=256, 
                                activation="relu"))
    model.add(Dropout(i))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(i))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    #compile the model using Adam as an optimizer and 
    #a binary cross entropy as a loss function
    print("[INFO] training network...")
    opt = Adam(lr=INIT_LR)
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
    callbacks = [ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=0.00001)] 
    history_model = model.fit(trainX, trainY, epochs=EPOCHS, batch_size=BATCH, callbacks=callbacks, verbose=1)
    models.append(model)
    # evaluate the model using train and test data
    _, train_acc = model.evaluate(trainX, trainY, verbose=0)
    _, test_acc = model.evaluate(testX, testY, verbose=0)
    print('Param: %f, Train: %.3f, Test: %.3f' % (param, train_acc, test_acc))
    all_train.append(train_acc)
    all_test.append(test_acc)

Plotting train and test accuracies depending on Dropout parameter

In [ ]:
plt.semilogx(parameters, all_train, label='train', marker='o')
plt.semilogx(parameters, all_test, label='test', marker='o')
plt.legend()
plt.show()

In [ ]:
all_train_array = np.array(all_train)
all_test_array = np.array(all_test)
accuracy_frame = pd.DataFrame({
                        'parameter' : parameters,
                        'accuracy_on_train': all_train_array*100, 
                         'accuracy_on_test': all_test_array*100,
                         'accuracy_difference': abs((all_train_array - all_test_array)*100)
                              })
acc=accuracy_frame.sort_values(by=['accuracy_difference'])
acc

In [ ]:
predict = (models[3].predict(testX) > 0.5).astype("int32")
print(classification_report(testY,predict))
print(confusion_matrix(testY, predict))

In [ ]:
print('Accuracy:',(accuracy_score(testY, predict))*100)